Scala 作为一门函数式编程语言，下划线这个符号在不同的场景下具有不同的含义，绕晕了不少初学者。下划线这个特殊符号无形中增加Scala的入门难度，下面来总结下 Scala 中下划线的用法。

## 用于替换Java的等价语法
由于大部分的Java关键字在Scala中拥有了新的含义，所以一些基本的语法在Scala中稍有变化。

### 导入通配符
*在Scala中是合法的方法名，所以导入包时要使用_代替。

```scala
//Java
import java.util.*;

//Scala
import java.util._
```

### 可变变量默认值
**只适用于var声明的变量。**

Java中类成员可以不赋初始值，编译器会自动帮你设置一个合适的初始值：
```java
class Foo{
     //String类型的默认值为null
     private String s;
}
```
而在Scala中必须要显式指定，如果你比较懒，可以用_让编译器自动帮你设置初始值：
```scala
class Foo{
    //String类型的默认值为null
    var s: String = _
}
```

该语法不但适用于类成员，也适用于局部变量。
```scala
//String类型的默认值为null
var s: String = _
```

### 解包传入可变参数
Java声明可变参数如下：
```java
public static void printArgs(String ... args){
    for(String arg: args){
        System.out.println(arg);
    }
}
```
调用方法如下：
```java
//传入两个参数
printArgs("a", "b");
//也可以传入一个数组
printArgs(new String[]{"a", "b"});
```

在Java中可以直接将数组传给printArgs方法，但是在Scala中，你必须要明确的告诉编译器，你是想将集合作为一个独立的参数传进去，还是想将集合的元素传进去。如果是后者则要借助下划线组合`:_*`：
```scala
def printArgs(args:String*) = for(arg <- args) println(arg)

//传入两个参数
printArgs("a", "b");
//解包
printArgs(List("a", "b"):_*)
```

### 类型通配符
Java的泛型系统有一个通配符类型，例如`List<?>`，任意的`List<T>`类型都是`List<?>`的子类型，如果我们想编写一个可以打印所有List类型元素的方法，可以如下声明：
```java
public static void printList(List<?> list){
    for(Object elem: list){
        System.out.println(elem);
    }
}
```

对应的Scala版本为：
```scala
def printList(list: List[_]): Unit ={
   list.foreach(elem => println(elem + " "))
}
```

## 模式匹配
模式匹配的正常使用和利用模式匹配赋值都可以使用_。

### 一般匹配
```scala
x match {
    case 1 => "one"
    case 2 => "two"
    case _ => "anything other than one and two"
}
```

### 匹配集合元素
```scala
expr match {
  case List(1,_,_) => " a list with three element and the first element is 1"
  case List(_*)  => " a list with zero or more elements "
  case Map[_,_] => " matches a map with any key type and any value type "
  case _ =>
}

//匹配以0开头，长度为三的列表
expr match {
  case List(0, _, _) => println("found it")
  case _ =>
}

//匹配以0开头，长度任意的列表
expr match {
  case List(0, _*) => println("found it")
  case _ =>
}

//匹配元组元素
expr match {
  case (0, _) => println("found it")
  case _ =>
}

//将首元素赋值给head变量
val List(head, _*) = List("a")
```

## Scala特有语法
### 访问Tuple元素
```scala
val t = (1, 2, 3)
println(t._1, t._2, t._3)
```

### 临时参数
```scala
List(1, 2, 3) foreach { _ => println("Hi") }    //List(1, 2, 3) foreach { t => println("Hi") }
```

### 通配模式
```scala
Some(5) match { case Some(_) => println("Yes") }

match {
     case List(1,_,_) => " a list with three element and the first element is 1"
     case List(_*)  => " a list with zero or more elements "
     case Map[_,_] => " matches a map with any key type and any value type "
     case _ =>
 }

val (a, _) = (1, 2)
for (_ <- 1 to 10)
```

### 通配导入
```scala
// Imports all the classes in the package matching
import scala.util.matching._

// Imports all the members of the object Fun (static import in Java).
import com.test.Fun._
```

### 隐藏导入
```scala
// Imports all the members of the object Fun but renames Foo to Bar
//导入所有，Foo重命名为Bar
import com.test.Fun.{ Foo => Bar , _ }

// Imports all the members except Foo. To exclude a member rename it to _
//导入所有，忽略Foo
import com.test.Fun.{ Foo => _ , _ }
```

### 简写函数
如果函数的参数在函数体内只出现一次，则可以使用下划线代替：
```scala
val nums = List(1,2,3,4,5,6,7,8,9,10)

nums map (_ + 2)
nums sortWith(_>_)
nums filter (_ % 2 == 0)
nums reduceLeft(_+_)
nums reduce (_ + _)
nums reduceLeft(_ max _)
nums.exists(_ > 5)
nums.takeWhile(_ < 8)
```

还有下面形式的函数定义：
```scala
val f1 = (_: Int) + (_: Int)
//等价于
val f2 = (x: Int, y: Int) => x + y

( (_: Int) + (_: Int) )(2,3)
```

### 定义部分应用函数（partially applied function）
我们可以为某个函数只提供部分参数进行调用，返回的结果是一个新的函数，即部分应用函数。因为只提供了部分参数，所以部分应用函数也因此而得名。
```scala
def sum(a: Int, b: Int, c: Int) = a + b + c
val b = sum(1, _: Int, 3)
b: Int => Int = <function1>
b(2) //6
```

错误的语法：
```scala
val b = sum(1, _, 3)
```
原因是，函数可能会有重载。

### 将方法转换成函数
Scala中方法和函数是两个不同的概念，方法无法作为参数进行传递(似乎是可以当做参数传递的)，也无法赋值给变量，但是函数是可以的。在Scala中，利用下划线可以将方法转换成函数：
```scala
//定义的是函数
val f = (x: Int, y: Int) => x + y
f: (Int, Int) => Int = <function2>

//函数可以直接赋值
val f1 = f
f1: (Int, Int) => Int = <function2>

//函数加了下划线反而变成了一个无参的高阶函数
val f1 = f _
f1: () => (Int, Int) => Int = <function0>

//定义的是方法（REPL中有默认的全局对象）
def ff(x: Int, y: Int) = x + y
defined function ff

//方法直接赋值就报错
val f1 = ff
You can make this conversion explicit by writing `ff _` or `ff(_,_)` instead of `ff`.

//方法加了下划线转换成函数
val f1 = ff _
f1: (Int, Int) => Int = <function2>
```